📒 1. Personalized Offers Notebook

In [5]:
import redis, numpy as np, json
from redis.commands.search.query import Query

r = redis.Redis(host='localhost', port=6379, decode_responses=True)

from redis.commands.search.field import VectorField, TextField
from redis.commands.search.index_definition import IndexDefinition, IndexType

schema = [
    TextField("$.category", as_name="category"),
    VectorField("$.embedding", "HNSW", {
        "TYPE": "FLOAT32",
        "DIM": 384,
        "DISTANCE_METRIC": "COSINE"
    }, as_name="embedding")
]

r.ft("idx:offers").create_index(schema, definition=IndexDefinition(prefix=["offer:"], index_type=IndexType.JSON))

# Add new offer
offer_id = "offer:001"
offer_data = {
    "category": "lifestyle",
    "embedding": list(np.random.rand(384))
}
r.execute_command("JSON.SET", offer_id, "$", json.dumps(offer_data))

# Semantic Search
query_vector = np.random.rand(384).astype(np.float32).tobytes()
q = Query("*=>[KNN 3 @embedding $vec AS score]")\
    .sort_by("score").return_fields("category", "score").dialect(2)
r.ft("idx:offers").search(q, query_params={"vec": query_vector})


Result{1 total, docs: [Document {'id': 'offer:001', 'payload': None, 'score': '0.244075298309', 'category': 'lifestyle'}]}

📒 2. Fraud Detection Notebook

In [7]:
import redis, numpy as np, json
from redis.commands.search.query import Query
# Connect to Redis
r = redis.Redis(host='localhost', port=6379, decode_responses=True)

from redis.commands.search.field import VectorField, TextField, NumericField
from redis.commands.search.index_definition import IndexDefinition, IndexType
import redis



# Define constants
INDEX_NAME = "idx:txns"
VECTOR_DIM = 128  # update based on your embedding size

# Drop index if it exists (optional during dev)
try:
    r.ft(INDEX_NAME).dropindex(delete_documents=False)
except Exception as e:
    print(f"Index did not exist or failed to drop: {e}")

# Define index schema
schema = (
    TextField("$.txn_id", as_name="txn_id"),
    NumericField("$.amount", as_name="amount"),
    VectorField("$.embedding", "FLAT", {
        "TYPE": "FLOAT32",
        "DIM": VECTOR_DIM,
        "DISTANCE_METRIC": "COSINE",
        "INITIAL_CAP": 1000
    }, as_name="embedding")
)

# Define the index definition
definition = IndexDefinition(prefix=["txn:"], index_type=IndexType.JSON)

# Create the index
r.ft(INDEX_NAME).create_index(fields=schema, definition=definition)

# Insert a transaction
txn_id = "txn:9001"
txn_data = {
    "type": "POS",
    "amount": 879.55,
    "timestamp": 1723456888,
    "embedding": list(np.random.rand(384))
}
r.execute_command("JSON.SET", txn_id, "$", json.dumps(txn_data))

# Query for similar transactions
query_vector = np.random.rand(384).astype(np.float32).tobytes()
q = Query("@type:{POS} => [KNN 5 @embedding $vec AS score]")\
    .sort_by("score").return_fields("amount", "score").dialect(2)
r.ft("idx:txns").search(q, query_params={"vec": query_vector})


Index did not exist or failed to drop: Unknown Index name


Result{0 total, docs: []}

📒 3. Customer Support Matching Notebook

In [14]:
import redis, numpy as np, json
from redis.commands.search.query import Query
# Connect to Redis
r = redis.Redis(host='localhost', port=6379, decode_responses=True)

from redis.commands.search.field import VectorField, TextField, NumericField
from redis.commands.search.index_definition import IndexDefinition, IndexType
import redis
# Parameters
INDEX_NAME = "idx:qna"
VECTOR_DIM = 384  # or whatever dimension your vector has

# Optional: Drop if it already exists
try:
    r.ft(INDEX_NAME).dropindex(delete_documents=False)
except:
    pass

# Define the schema
schema = (
    TextField("$.question", as_name="question"),
    TextField("$.answer", as_name="answer"),
    VectorField("$.embedding", "FLAT", {
        "TYPE": "FLOAT32",
        "DIM": VECTOR_DIM,
        "DISTANCE_METRIC": "COSINE",
        "INITIAL_CAP": 1000
    }, as_name="embedding")
)

# Define the index on JSON documents with key prefix "qna:"
definition = IndexDefinition(prefix=["qna:"], index_type=IndexType.JSON)

# Create the index
r.ft(INDEX_NAME).create_index(fields=schema, definition=definition)

qid = "qna:01"
qa = {
    "question": "How to block my card?",
    "answer": "You can block it instantly via mobile app.",
    "embedding": list(np.random.rand(384))
}
r.execute_command("JSON.SET", qid, "$", json.dumps(qa))

# Semantic query
query_vector = np.random.rand(384).astype(np.float32).tobytes()
q = Query("*=>[KNN 1 @embedding $vec AS score]")\
    .sort_by("score").return_fields("question", "answer", "score").dialect(2)
r.ft("idx:qna").search(q, query_params={"vec": query_vector})

Result{1 total, docs: [Document {'id': 'qna:01', 'payload': None, 'score': '0.22782510519', 'question': 'How to block my card?', 'answer': 'You can block it instantly via mobile app.'}]}